In [ ]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize, LabelBinarizer
import sys
sys.path.append('../..')
from modules.many_features import utils, constants

In [ ]:
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

#### The Datasets

In [ ]:
#train_df = pd.read_csv('../../final/data/train_set_noisy_6_missing_3.csv')
train_df = pd.read_csv('../../final/data/train_set_basic.csv')
train_df = train_df.fillna(-1)
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train.head()

In [ ]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]
X_test.head()

In [ ]:
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
y_test

#### Performance before normalizing

In [ ]:
from sklearn.svm import SVC
svm_model = SVC(kernel='linear', C=1, decision_function_shape='ovo', random_state=constants.SEED).fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
test_df_svm = pd.DataFrame()
test_df_svm['y_actual'] = y_test
test_df_svm['y_pred'] = y_pred_svm
test_df_svm.isna().sum()

In [ ]:
acc_svm, f1_svm, roc_auc_svm = utils.test(test_df_svm['y_actual'], test_df_svm['y_pred'])
acc_svm, f1_svm, roc_auc_svm

#### Normalizing X

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
mmc = MinMaxScaler()

In [ ]:
X_train_norm = mmc.fit_transform(X_train)
X_test_norm = mmc.transform(X_test)

In [ ]:
X_train_norm.shape, X_test_norm.shape

#### Support Vector Machine

In [ ]:
from sklearn.svm import SVC
svm_model = SVC(kernel='linear', C=1, decision_function_shape='ovo', random_state=constants.SEED).fit(X_train_norm, y_train)
y_pred_svm = svm_model.predict(X_test_norm)
test_df_svm = pd.DataFrame()
test_df_svm['y_actual'] = y_test
test_df_svm['y_pred'] = y_pred_svm
test_df_svm.isna().sum()

In [ ]:
success_rate_svm, success_df_svm = utils.success_rate(test_df_svm)
success_rate_svm

In [ ]:
acc_svm, f1_svm, roc_auc_svm = utils.test(test_df_svm['y_actual'], test_df_svm['y_pred'])
acc_svm, f1_svm, roc_auc_svm

#### Feed-forward Neural Network - Keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras import backend as K
import keras

In [ ]:
#keras.utils.set_random_seed(42)

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
def numerize_labels(pd_series):
    series_copy = pd_series.copy()
    series_copy = series_copy.map(class_dict)
    return series_copy

y_train_num = y_train
y_test_num = y_test

dummy_y_train = np_utils.to_categorical(y_train_num)
dummy_y_test = np_utils.to_categorical(y_test_num)
dummy_y_train.shape, dummy_y_test.shape

In [ ]:
model = Sequential()
model.add(Dropout(0.5, input_shape=(17,)))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', f1_m, precision_m, recall_m])
history = model.fit(X_train_norm, dummy_y_train, epochs=100, batch_size=1024, shuffle=True, validation_split=0.20, verbose=0)
#model.save('../models/baselines/synthentic_nn.h5')
loss, accuracy, f1, precision, recall = model.evaluate(X_test_norm, dummy_y_test, verbose=0)
loss, accuracy, f1, precision, recall

In [ ]:
#Alternatively
y_pred1_nn = model.predict(X_test_norm)
y_pred_nn = np.argmax(y_pred1_nn, axis=1)
np.unique(y_pred_nn)

In [ ]:
test_df_nn = pd.DataFrame()
test_df_nn['y_actual'] = y_test
test_df_nn['y_pred'] = y_pred_nn
test_df_nn.isna().sum()

In [ ]:
success_rate_nn, success_df_nn = utils.success_rate(test_df_nn)
success_rate_nn

In [ ]:
acc_nn, f1_nn, roc_auc_nn = utils.test(test_df_nn['y_actual'], test_df_nn['y_pred'])
acc_nn, f1_nn, roc_auc_nn